In [ ]:
# %% [markdown]
# # Step 1: Getting Trust Anchor Crentials for WineShop and Government
#
# - ledger intended to store **Identity Records** that describe a **Ledger Identity**
# - **Identity Records** are public data and may include Public Keys, Service Endpoints, Credential Schemas, Credential Definitions
# - Every **Identity Record** is associated with exactly one **DID** (Decentralized Identifier) that is gloablly unique and resolvable (via a ledger) without requiring centralized resolution authority
# - To maintain privacy each **Identity Owner** can own multiple DIDs
#
# ## DIDS
# 1. Verinym
#     - associated with **Legal Identity** of **Identity Owner**
#     - all parties should be able to verify that some DID is used by a Government to publish schemas for some document type
# 2. Pseudonym
#     - **Blinded Identifier** to maintain privacy in the context of ongoing digital relationship (**Connection**)
#     - if only one digital relationship: **Pairwise-Unique Identifier**
#
# ## NYM transaction
# - creation of a DID known to the ledger is an **Identity Record** itself
# - can be used for
#     - creation of new DIDs known to that
#     - setting and rotation of verification key
#     - setting and changing of roles
# - most important fields of NYM transaction
#     - ```dest```(target DID)
#     - ```role```(role of a use NYM record being created for)
#     - ```verkey``` (target verification key)
#
# ## DID verification key
# - allows person, organization or thing to verify that someone owns this DID as that person, organization or thing is the **only one** who knows corresponding signing key and any DID-related operations requiring signing with this key
#
# ## Trust Anchor
# - everyone who wants to publish on public permissioned ledger needs to get role of **Trust Anchor** on the ledger
# - person or organization ledger already knows about
# - **first step towards being able to place transactions on the ledger involves getting role of Trust Anchor on the ledger**
# - becoming Trust Anchor requires contacting person or organization who already has Trust Anchor role on the ledger
#
# ## Stewards
# - all Stewars are automatically **Trust Anchors**
#
# WineShop needs to get the role of a **Trust Anchor** on the ledger to create Verinyms and Pairwise-Unique-Identifiers to provide service to customers

# %% [markdown]
# # Step 2: Connecting to the Indy Nodes Pool
#
# ## Code of the **Steward's** agent
# ### Pool Configuration
# - To write and read ledger's transactions after gaining proper role, connection to Indy nodes pool is required
# - Set up connection to different existing pools (e.g. Sovrin pool), pool configuration needs to be set up
# - list of nodes in the pool is stored in the ledger as NODE transactions
# - Libindy allows you to restore actual list of NODE transactions by a few known transactions (genesis transactions)
# - Each **Pool Configuration** defined as pair of pool configuration **name** and pool configuration **JSON**
# - most important field in pool configuration: **path to the file with list of genesis transactions**
#
# ```pool.create_pool_ledger_config``` call allows you to create a named pool configuration
# - After the pool configuration is created we can connect to the nodes pool that this configuration describes by ```calling pool.open_pool_ledger```
#     - This call returns the pool handle that can be used to reference this opened connection in future libindy calls

# %%
#pip install indy-sdk

# %%
import asyncio
import time
import json

from indy import anoncreds, didDID, ledger, pool, wallet, IndyError
from indy.error import ErrorCode

# %%
await pool.set_protocol_version(2)

# pool configuration name
pool_ = {'name': 'pool1'}
# path to the file with list of genesis transactions
pool_['genesis_txn_path'] = get_pool_
genesis_txn_path(pool_['name'])
# pool configuration JSON
pool_['config'] = json.dumps({"genesis_txn": str(pool_['genesis_txn_path'])})
# create named pool configuration
await pool.create_pool_ledger_config(pool_['name'], pool_['config'])
# connect to nodes pool
pool_['handle'] = await pool.open_pool_ledger(pool_['name'], None)

# %% [markdown]
# # Step 3: Getting the ownership for Steward's Verinym
# -  Steward's agent should get ownership for the DID that has corresponding NYM transactions with Steward role on the ledger
#
# ## Seed values
# - for random number generator used to generate keys for NYMs
# - restore signing keys for DIDs on Steward's agent side and as a result get DID ownership
#
# ## Libindy Wallet
# - secure storage for crypto materials like DIDs, keys, etc
# - To store **Steward's** DID and corresponding signkey, agent should create named wallet first by calling ```wallet.create_wallet```

# %% [markdown]
#

# %%
# Steward Agent
steward = {
    'name': "Sovrin Steward",
    'wallet_config': json.dumps({'id': 'sovrin_steward_wallet'}),
    'wallet_credentials': json.dumps{('key': 'steward_wallet_key'},
    'pool': pool_['handle'],
    'seed': '000000000000000000000000Steward1'
}

# create named wallet to store Steward's DID and corresponding signkey
await wallet.create_wallet(steward['wallet_config'], steward['wallet_credentials'])
print(steward['wallet_config'])


# %%


# %% [markdown]
#